<a href="https://colab.research.google.com/github/NUELBUNDI/PDS_PROJECT/blob/main/mpesa_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [234]:
import io
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [235]:
url="https://raw.githubusercontent.com/NUELBUNDI/PDS_PROJECT/main/mpesa_payment.csv"
df=pd.read_csv(url,encoding = "ISO-8859-1")

In [236]:
df.head(1)

,Posting Date,Source Code,Dimension Set ID,Business Unit Code,Department Code,Bal. Account No.,Amount (LCY),Bal. Account Type,Document Type,Document No.,User ID,Bank Account No.,Description,Amount,Drep Code,Payment Method,Credit Card Code,Credit Card No.,External Document No.,Drawer Name,Open,Entry No.
0,28/01/2022,GENJNL,7985,21010,1070,U100110,"-9,550.00",G/L Account,,MED032911,PGAKONO,B330020,MEDICAL REFUND - JOAN SHARON MUYERA,"-9,550.00",31070,NaN,NaN,NaN,QAS9ADKGG9,NaN,True,7059713


In [237]:
df.to_csv("data.csv")

##Cleaning of data


In [238]:
class  clean_data:

  def __init__(self,data_frame):
    self.data_frame=data_frame

  def clean_data(self):
      # rename the columns name
      col_name=[]
      new_colname=[]
      for colname in self.data_frame.columns:
        col_name.append(colname)
        output=colname.replace(' ','').replace('.','').lower()
        new_colname.append(output)
      col_rename_dict={col_name[i]:new_colname[i] for i in range(len(new_colname))}
      self.data_frame.rename(columns=col_rename_dict,inplace=True)
      # Subset the data
      df2=self.data_frame[['postingdate','sourcecode','documentno','externaldocumentno','amount','businessunitcode','departmentcode','balaccountno','description','userid']]

      return df2                     

obj=clean_data(df)

df2=obj.clean_data()


In [239]:
def convert_amount(val):
  res=val.replace(',','').replace('-','')
  return res
df2['amount']=df2['amount'].apply(lambda x :convert_amount(x)).astype('float')
df=df2

df.head(2)


,postingdate,sourcecode,documentno,externaldocumentno,amount,businessunitcode,departmentcode,balaccountno,description,userid
0,28/01/2022,GENJNL,MED032911,QAS9ADKGG9,9550.0,21010,1070,U100110,MEDICAL REFUND - JOAN SHARON MUYERA,PGAKONO
1,28/01/2022,GENJNL,MED032948,QAS4ADIUY4,8991.0,22070,1080,U100110,MEDICAL REFUND - ABDIKADIR IBRAHIM DIRIYE,PGAKONO


In [240]:
pd.set_option('mode.chained_assignment', None)

df['businessunitcode']=df['businessunitcode'].astype('str')
df['departmentcode']=df['departmentcode'].astype('str')

In [254]:
# Drop all reversed Entries

df_reversed=df[df['sourcecode']=='REVERSAL']

doc_no_reversed=[]
for x in df_reversed['documentno']:
  doc_no_reversed.append(x)

rows_to_drop=df.query('documentno==@doc_no_reversed')
indexName=rows_to_drop.index
df.drop(indexName, inplace=True)

df['sourcecode'].unique()


array(['GENJNL', 'PAYMENTJNL', 'JOBGLJNL', 'CASHRECJNL'], dtype=object)

In [252]:
# Drop all sweep posted to that account

drop_mpesa_sweep=df.query('balaccountno=="B302010*" | balaccountno=="B301010"').index
df.drop(drop_mpesa_sweep,inplace=True)

In [255]:
df['type']=df['documentno'].str[:3]

for x in df['type'].unique():
    print(x,end=',')


MED,PC0,FLT,CRE,GJN,JJN,R00,

In [243]:
for x in df['type'].unique():
    print(x,end=',')

MED,PC0,FLT,CRE,GJN,JJN,R00,

In [244]:
mask=df['type'].str.contains('MED|PC0|FLT|CRE|GJN')
df[mask]

,postingdate,sourcecode,documentno,externaldocumentno,amount,businessunitcode,departmentcode,balaccountno,description,userid,type
0,28/01/2022,GENJNL,MED032911,QAS9ADKGG9,9550.0,21010,1070,U100110,MEDICAL REFUND - JOAN SHARON MUYERA,PGAKONO,MED
1,28/01/2022,GENJNL,MED032948,QAS4ADIUY4,8991.0,22070,1080,U100110,MEDICAL REFUND - ABDIKADIR IBRAHIM DIRIYE,PGAKONO,MED
2,28/01/2022,GENJNL,MED032947,QAS2ADHTGC,1000.0,24010,1080,U100110,MEDICAL REFUND - ARTHUR MACHARIA KIONI,PGAKONO,MED
3,28/01/2022,GENJNL,MED032946,QAS8ADHFTE,15000.0,21010,5010,U100110,MEDICAL REFUND - BRENDA OWINO,PGAKONO,MED
4,28/01/2022,PAYMENTJNL,PC015778,NaN,50000.0,21010,4150,1019,GADGET PURCHASE AS PER POLICY,PGAKONO,PC0
...,...,...,...,...,...,...,...,...,...,...,...
9826,01/09/2021,GENJNL,FLT003149,NaN,98.0,23100,1080,B361010,Float Reimburse - Agnes Wambui Macharia,FMUEMA,FLT
9827,01/09/2021,GENJNL,CREF001285,PI15CFFWA1,3100.0,23080,1080,101544,BALANCE IN ACCOUNT AFTER EXCHANGING DDA1200 WI...,LBUNDI,CRE
9828,01/09/2021,GENJNL,CREF001284,PI14CFFCX2,61712.0,21010,1080,11602,REFUND OF AMOUNT FOR RETURNED PUMP LESS HNDLIN...,LBUNDI,CRE
9829,01/09/2021,GENJNL,CREF001283,PI13CFF0JR,15660.0,22010,1080,103032,REFUND FOR ITEM RETURNED - REF Credit Note No ...,LBUNDI,CRE


In [245]:

# df.query('balaccountno=="P100500" & type=="GJN"')
# pattern='casual payment'
# # mask1=

# mask=df['description'].str.contains(pattern,case=False,na=False)

# df[mask]

In [246]:
# df.to_csv('excel.csv')